돌릴 때 T4GPU, 고용량RAM

# 패키지 import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from pandas import concat
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import log_loss,confusion_matrix,classification_report,roc_curve,auc

import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from scipy import sparse
%matplotlib inline
seed = 42
import os

In [ ]:
import keras
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

from keras.layers import Bidirectional
from keras.layers import Dropout
from keras.layers import GlobalMaxPooling1D
from keras.regularizers import l2
from keras.layers import concatenate
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.layers import Concatenate
from keras.layers import Input

from keras.models import Model
from keras.optimizers import Nadam
from keras.layers import BatchNormalization
from keras.callbacks import ReduceLROnPlateau

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# 데이터 로드

In [ ]:
# 경로 설정

products = pd.read_csv('/content/drive/MyDrive/DIMA 프로젝트 1조/EC21 dataset/products_no_china_no_token_v2.csv')

In [ ]:
# 코랩에서

pip install dill

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

# import 해야하는 모듈
import dill
import pandas as pd
from bs4 import BeautifulSoup
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import re

# 전처리 함수
- 피클 파일 없으면 함수 실행하고 저장 -> 로드 하기
- 피클 파일 있으면 바로 로드 하기

In [ ]:
def preprocess_text(df, col_names=['CATALOG_NM', 'CATALOG_DESC']):

    # html태그 확인
    def has_html_tags(col):
        soup = BeautifulSoup(col, 'html.parser')
        if bool(soup.find()) :
            return soup.get_text()
        else:
            return col

    remove_punct_dict = {ord(punct):' ' for punct in string.punctuation}
    lemmar = WordNetLemmatizer()

    # 문장 입력받음 -> 소문자로 변환 -> 구두점 처리 -> 단어로 토큰 -> 불용어 제거 -> 어근 추출
    def len_normalize(text):
        tokens = nltk.word_tokenize(text.lower().translate(remove_punct_dict))
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words]
        return [lemmar.lemmatize(token) for token in tokens]


    # 컬럼 선택
    df = df[col_names]

    # DESC 컬럼에 결측치 존재하는 경우
    if df[col_names[1]].isna().sum()!=0:
        df.fillna('',inplace=True)

    # DESC 컬럼 개행문자 제거
    df[col_names[1]] = df[col_names[1]].str.replace('\r',' ').str.replace('\n',' ').str.replace('\\',' ')

    # DESC 컬럼 태그 확인 및 제거
    df[col_names[1]] = df[col_names[1]].apply(has_html_tags)

    # 태그 지우고 공백인 경우, 상품명으로 대체
    mask = df[col_names[1]] == ''  # boolean Series 생성
    df.loc[mask, col_names[1]] = df.loc[mask, col_names[0]]  # 조건을 만족하는 행만 선택하여 업데이트

    # 불완전 태그 제거
    pattern = re.compile(r'<[a-z]\s+[\d\W\s\w]*\s*=?\s?[0-9]*[\d\W\s\w]')
    df[col_names[1]] = df[col_names[1]].apply(lambda x: re.sub(pattern, ' ', x))

    # 불완전 태그 제거한 후 공백인 경우 NM컬럼 값으로 대체
    df[col_names[1]] = df.apply(lambda x: x[col_names[0]] if x[col_names[1]].isspace() else x[col_names[1]], axis=1)

    # 클랜징
    for i in range(len(col_names)):
        df[col_names[i]] = df[col_names[i]].apply(len_normalize)

    return df

In [ ]:
# 피클 파일에 저장
with open('preprocessing_function.pkl', 'wb') as file:
    dill.dump(preprocess_text, file)

In [ ]:
# 함수 로드
with open('preprocessing_function.pkl', 'rb') as file:
    loaded_function = dill.load(file)

# 데이터 프레임 전체 개수 줄이기

In [ ]:
df_0 = products[products['judge'] == 0].sample(n=90000, random_state=42)
df_1 = products[products['judge'] == 1].sample(n=120000, random_state=42)

# Concatenate the DataFrames
df = pd.concat([df_0, df_1])

In [ ]:
# Assuming df1 and y are already defined
y = df['judge']

# 피클파일 쓰는법
- 변수 설정은 알아서

In [ ]:
df = loaded_function(products, col_names=['CATALOG_NM', 'CATALOG_DESC'])

# 모델
- df['CATEGORYM_ID'] 를 바꾸기

In [ ]:
# Step 1: Tokenization
# 단계 1: 토큰화
tokenizer = Tokenizer()
tokenizer.fit_on_texts(pd.concat([df['CATALOG_DESC'], df['CATALOG_NM'],df['CATEGORYM_ID'].astype(str)]))

# Vocabulary size
vocab_size = len(tokenizer.word_index) + 1


# Step 2: Integer Encoding
sequences_desc = tokenizer.texts_to_sequences(df['CATALOG_DESC'])
sequences_nm = tokenizer.texts_to_sequences(df['CATALOG_NM'])
sequences_new = tokenizer.texts_to_sequences(df['CATEGORYM_ID'].astype(str))

max_length_desc = max(len(seq) for seq in sequences_desc)
max_length_nm = max(len(seq) for seq in sequences_nm)
max_length_new = max(len(seq) for seq in sequences_new)

x_desc = pad_sequences(sequences_desc, maxlen=max_length_desc, padding='post')
x_nm = pad_sequences(sequences_nm, maxlen=max_length_nm, padding='post')
x_new = pad_sequences(sequences_new, maxlen=max_length_new, padding='post')


# Split the data into train and validation sets for 'NEW_INPUT'
X_train_desc, X_test_desc, X_train_nm, X_test_nm, X_train_new, X_test_new, y_train, y_test = train_test_split(
    x_desc, x_nm, x_new, y, test_size=0.3, random_state=42)

# Split the remaining data for 'NEW_INPUT' into validation and test sets
X_val_desc, X_test_desc, X_val_nm, X_test_nm, X_val_new, X_test_new, y_val, y_test = train_test_split(
    X_test_desc, X_test_nm, X_test_new, y_test, test_size=0.5, random_state=40)


# Step 2: Define LSTM model for 'CATALOG_DESC'
model_desc = Sequential()
model_desc.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=max_length_desc))
model_desc.add(LSTM(units=100))
model_desc.add(Dropout(0.2))

# Step 3: Define LSTM model for 'CATALOG_NM'
model_nm = Sequential()
model_nm.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=max_length_nm))
model_nm.add(LSTM(units=100))
model_nm.add(Dropout(0.2))

# Step 4: Define LSTM model for 'NEW_INPUT'
model_new = Sequential()
model_new.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=max_length_new))
model_new.add(LSTM(units=100))
model_new.add(Dropout(0.2))


# Step 5: Concatenate sequences using Keras Concatenate layer
concat_layer = Concatenate()([model_desc.output, model_nm.output, model_new.output])

# Step 6: Add additional Dense layer after concatenation if needed
merged = Dense(units=50, activation='relu')(concat_layer)

# Step 7: Output layer
output_layer = Dense(units=1, activation='sigmoid')(merged)

# Step 8: Create a new model with the concatenated layers
model_combined = Model(inputs=[model_desc.input, model_nm.input, model_new.input], outputs=output_layer)

# Step 9: Compile the model
model_combined.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 10: Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=3)

# Step 11: Checkpoint
current_directory = os.getcwd()
models_folder_path = os.path.join(current_directory, 'Models')
os.makedirs(models_folder_path, exist_ok=True)
check_point = ModelCheckpoint(
    filepath=os.path.join(models_folder_path, '{epoch:02d}-{val_accuracy:.3f}.hdf5'),
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True
)

# Step 12: Train the model using separate validation data
model_combined.fit(
    [X_train_desc, X_train_nm, X_train_new],
    y_train,
    epochs=20,
    batch_size=32,
    validation_data=([X_val_desc, X_val_nm, X_test_new], y_val),  # Use X_test_new for validation data
    verbose=1,
    callbacks=[early_stopping, check_point]
)

# 평가 1
- 평가 지표
- load_model에 쓸 모델 경로 설정해두기
- threshold = 0.5


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
from keras.models import load_model

# load model
loaded_model = load_model('/content/drive/MyDrive/Colab Notebooks/better.hdf5')

# evaluate-score
score = loaded_model.evaluate([X_test_desc, X_test_nm, X_test_new], y_test ,verbose=1)

# Obtain predictions on the validation set
y_pred = loaded_model.predict([X_val_desc, X_val_nm, X_val_new])

# Convert probabilities to binary predictions using a threshold (e.g., 0.5)
threshold = 0.5
y_pred_binary = (y_pred > threshold).astype(int)

# Evaluate the model
conf_matrix = confusion_matrix(y_val, y_pred_binary)

accuracy = accuracy_score(y_val, y_pred_binary)
precision = precision_score(y_val, y_pred_binary)
recall = recall_score(y_val, y_pred_binary)
f1 = f1_score(y_val, y_pred_binary)
roc_auc = roc_auc_score(y_val, y_pred)

# Print the metrics
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'ROC-AUC Score: {roc_auc:.4f}')

# Print the confusion matrix
print('\n==== Confusion Matrix ====')
print(conf_matrix)

# Print the classification report
print('\n==== Classification Report ====')
print(classification_report(y_val, y_pred_binary))

평가 2 maxlen 잡기

In [ ]:
print('sequences_desc:' , sequences_desc)
print('sequences_nm:' , sequences_nm)
print('sequences_new:' , sequences_new)

# 평가 2
- gpt 코드
- threshold = 0.5
- maxlen 변경하기

In [ ]:
# Assuming new_data is your DataFrame with the new data
# Tokenize and pad the new data
sequences_new_desc = tokenizer.texts_to_sequences(df['CATALOG_DESC'])
sequences_new_nm = tokenizer.texts_to_sequences(df['CATALOG_NM'])
sequences_new_new = tokenizer.texts_to_sequences(df['CATALOG_NM'])


# Pad the sequences based on the determined maximum lengths
x_new_desc = pad_sequences(sequences_new_desc, maxlen=5435, padding='post')
x_new_nm = pad_sequences(sequences_new_nm, maxlen=31, padding='post')
x_new_new = pad_sequences(sequences_new_nm, maxlen=31, padding='post')


# Predict on the new data
y_pred = loaded_model.predict([x_new_desc, x_new_nm, x_new_new])

# Apply threshold for binary classification
threshold = 0.5
y_pred_binary = (y_pred > threshold).astype(int)

# Get the indices corresponding to the original data
original_indices = df.index.tolist()

# Create a DataFrame with indices and corresponding predictions
result_df = pd.DataFrame({'Original_Index': original_indices, 'Predicted_Label': y_pred_binary.flatten()})